In [4]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer

In [13]:
with open('../data/preprocessed_X_visit_over3.pkl', 'rb') as f:
    data_x = pickle.load(f)

with open('../data/preprocessed_y_visit_over3.pkl', 'rb') as f: 
    labels = pickle.load(f)

In [73]:
top100_list = pickle.load(open('../new_data/top_100_list.pkl', 'rb'))
top10_list = pickle.load(open('../new_data/top_10_list.pkl', 'rb'))
top20_list = pickle.load(open('../new_data/top_20_list.pkl', 'rb'))
top40_list = pickle.load(open('../new_data/top_40_list.pkl', 'rb'))
top60_list = pickle.load(open('../new_data/top_60_list.pkl', 'rb'))
top80_list = pickle.load(open('../new_data/top_80_list.pkl', 'rb'))
bot10_list = pickle.load(open('../new_data/bot_10_list.pkl', 'rb'))
bot20_list = pickle.load(open('../new_data/bot_20_list.pkl', 'rb'))
bot40_list = pickle.load(open('../new_data/bot_40_list.pkl', 'rb'))
bot60_list = pickle.load(open('../new_data/bot_60_list.pkl', 'rb'))
bot80_list = pickle.load(open('../new_data/bot_80_list.pkl', 'rb'))

In [74]:
len(bot20_list)

20

In [75]:
mlb_t100 = MultiLabelBinarizer()
mlb_t100.fit([top100_list])

mlb_t10 = MultiLabelBinarizer()
mlb_t10.fit([top10_list])
mlb_t20 = MultiLabelBinarizer()
mlb_t20.fit([top20_list])
mlb_t40 = MultiLabelBinarizer()
mlb_t40.fit([top40_list])
mlb_t60 = MultiLabelBinarizer()
mlb_t60.fit([top60_list])
mlb_t80 = MultiLabelBinarizer()
mlb_t80.fit([top80_list])

mlb_b10 = MultiLabelBinarizer()
mlb_b10.fit([bot10_list])
mlb_b20 = MultiLabelBinarizer()
mlb_b20.fit([bot20_list])
mlb_b40 = MultiLabelBinarizer()
mlb_b40.fit([bot40_list])
mlb_b60 = MultiLabelBinarizer()
mlb_b60.fit([bot60_list])
mlb_b80 = MultiLabelBinarizer()
mlb_b80.fit([bot80_list])

MultiLabelBinarizer()

In [76]:
import warnings
warnings.filterwarnings('ignore')

In [77]:
new_labels = {}
for key, label_dict in labels.items():
    top10_label_bin = mlb_t10.transform([label_dict['top100_label']])
    top20_label_bin = mlb_t20.transform([label_dict['top100_label']])
    top40_label_bin = mlb_t40.transform([label_dict['top100_label']])
    top60_label_bin = mlb_t60.transform([label_dict['top100_label']])
    top80_label_bin = mlb_t80.transform([label_dict['top100_label']])
    bot10_label_bin = mlb_b10.transform([label_dict['top100_label']])
    bot20_label_bin = mlb_b20.transform([label_dict['top100_label']])
    bot40_label_bin = mlb_b40.transform([label_dict['top100_label']])
    bot60_label_bin = mlb_b60.transform([label_dict['top100_label']])
    bot80_label_bin = mlb_b80.transform([label_dict['top100_label']])
    
    p_label = {'origin_y': label_dict['origin_y'],
               'top100_label': label_dict['top100_label'],
               'top100_label_bin': label_dict['top100_label_bin'], 
               'top10_label_bin': top10_label_bin,
               'top20_label_bin': top20_label_bin,
               'top40_label_bin': top40_label_bin,
               'top60_label_bin': top60_label_bin,
               'top80_label_bin': top80_label_bin,
               'bot10_label_bin': bot10_label_bin,
               'bot20_label_bin': bot20_label_bin,
               'bot40_label_bin': bot40_label_bin,
               'bot60_label_bin': bot60_label_bin,
               'bot80_label_bin': bot80_label_bin}
    new_labels[key] = p_label

In [78]:
with open('../data/preprocessed_y_visit_over3_new.pkl', 'wb') as f: 
    pickle.dump(new_labels, f)
f.close()

In [82]:
labels = new_labels

In [83]:
visit_gt2t_new = {}
for k, v in data_x.items():
    # print(k, v)
    visit_new = {}
    for v_k, v_v in v.items():
        total_list = list(set(v_v['diagnoses'])) + list(set(v_v['procedures'])) + v_v['drugs']
        # visit_new[v_k] = v_v
        if len(total_list) > 1:
            visit_new[v_k] = {'diagnoses': None, 'procedures':None, 'drugs': None, 'admitdate':None}
            visit_new[v_k]['diagnoses'] = list(set(v_v['diagnoses'])) 
            visit_new[v_k]['procedures'] = list(set(v_v['procedures'])) 
            visit_new[v_k]['drugs'] = list(set(v_v['drugs'])) 
            visit_new[v_k]['admitdate'] = v_v['admitdate']
    visit_gt2t_new[k] = visit_new

In [84]:
adm = pd.read_csv('../data/admissions.csv.gz', compression='gzip', usecols=['subject_id', 'hadm_id', 'admittime', 'admission_type', 'race'])
patients = pd.read_csv('../data/patients.csv')
patients = patients[['subject_id', 'gender','anchor_age','anchor_year']]
patients['yob']= patients['anchor_year'] - patients['anchor_age']

In [85]:
visit_gt2t_last = {}
for k, v in visit_gt2t_new.items():
    # break
    if len(v) >= 2:
        visit_gt2t_last[k] = v

In [86]:
## drug selection
dr_list = list()
dr_length = list()
for k, visit in visit_gt2t_new.items():
    for v_id, v in visit.items():
        dr_list.extend(v['drugs'])
        dr_length.append(len(v['drugs']))

In [87]:
max(dr_length)

276

In [88]:
drug_dict = dict(Counter(dr_list))
sorted_drug_dict = dict(sorted(drug_dict.items(), key=lambda item: item[1], reverse=False))

In [89]:
data_dict = {}
rm_patient_num =0
rm_patient_list = []

for p_id, v_data in tqdm(visit_gt2t_last.items()):
    visit_data = {'seq': [], 'data':[], 'time': [], 'diagnoses_top100': [], 'diagnoses_top100_len':[],
                  'gender': patients.loc[patients['subject_id'] == p_id, 'gender'].unique()[0],
                  'race': adm.loc[adm['subject_id'] == p_id, 'race'].unique()[0]}
    seg_idx = 0
    for v_id, record in v_data.items():
        record['visit_id'] = v_id
        record['admission_type'] = adm.loc[(adm['subject_id'] == p_id) & (adm['hadm_id'] == v_id), 'admission_type'].values[0]
        dt_obj = datetime.strptime(record['admitdate'], '%Y-%m-%d')
        age_at_adm = dt_obj.year - patients.loc[patients['subject_id'] == p_id, 'yob'].values[0]
        if age_at_adm < 0:
            age_at_adm = 90
        record['age'] = age_at_adm
        if len(record['drugs']) > 50:
            ## 약물 코드가 50개 이상이면 빈도수 기준 50개만 선택
            sorted_drugs = sorted(sorted_drug_dict, key=sorted_drug_dict.get, reverse=True)
            top100_drugs = [elem for elem in sorted_drugs if elem in record['drugs']][:50]
            seq_data = record['diagnoses'] + record['procedures'] + top100_drugs
        else:
            seq_data = record['diagnoses'] + record['procedures'] + record['drugs']
        
        top100_diagnoses = [d_code for d_code in record['diagnoses'] if d_code in top100_list]
        visit_data['diagnoses_top100'].append(top100_diagnoses)
        visit_data['diagnoses_top100_len'].append(len(top100_diagnoses))
        visit_data['seq'].append(seq_data)
        visit_data['data'].append(record)
        visit_data['time'].append(dt_obj)
        
    if 0 in visit_data['diagnoses_top100_len']:
        rm_patient_num += 1
        rm_patient_list.append(p_id)
        continue
    
    td_list = [(visit_data['time'][-1]- dt).days for dt in visit_data['time']]
    visit_data['timedelta'] = td_list
    visit_data['labels_origin'] = labels[p_id]['top100_label']
    visit_data['labels_top100_bin'] = labels[p_id]['top100_label_bin']
    visit_data['labels_top10_bin'] = labels[p_id]['top10_label_bin']
    visit_data['labels_top20_bin'] = labels[p_id]['top20_label_bin']
    visit_data['labels_top40_bin'] = labels[p_id]['top40_label_bin']
    visit_data['labels_top60_bin'] = labels[p_id]['top60_label_bin']
    visit_data['labels_top80_bin'] = labels[p_id]['top80_label_bin']
    visit_data['labels_bot10_bin'] = labels[p_id]['bot10_label_bin']
    visit_data['labels_bot20_bin'] = labels[p_id]['bot20_label_bin']
    visit_data['labels_bot40_bin'] = labels[p_id]['bot40_label_bin']
    visit_data['labels_bot60_bin'] = labels[p_id]['bot60_label_bin']
    visit_data['labels_bot80_bin'] = labels[p_id]['bot80_label_bin']
    data_dict[p_id] = visit_data

100%|██████████| 43096/43096 [05:59<00:00, 119.85it/s]


In [ ]:
len(data_dict)

In [91]:
with open('../new_data/data4pretrained_filter_drugs_add_multi_bin.pkl', 'wb') as f:
    pickle.dump(data_dict, f)

In [ ]:
with open('../new_data/data4pretrained_filter_drugs_add_multi_bin.pkl', 'rb') as f:
    data_dict = pickle.load(f)

In [ ]:
d_list = list()
p_list = list()
dr_list = list()
for k, visit in data_dict.items():
    for v in visit['data']:
        d_list.extend(v['diagnoses'])
        p_list.extend(v['procedures'])
        dr_list.extend(v['drugs'])
        
d_list = list(set(d_list))
p_list = list(set(p_list))
dr_list = list(set(dr_list))

In [94]:
len(d_list), len(p_list), len(dr_list)

(863, 738, 3487)

In [95]:
code2idx = {'padding':0}

idx = 1
for d in d_list:
    code2idx[d] = idx
    idx += 1

for p in p_list:
    code2idx[p] = idx
    idx += 1

dr_idx = 1
for dr in dr_list:  
    code2idx[dr] = idx
    idx += 1

code2idx['[CLS]'] = idx
code2idx['[SEP]'] = idx + 1
code2idx['[MASK]'] = idx + 2

In [96]:
len(code2idx)

5092

In [97]:
code2idx['[MASK]']

5091

In [98]:
with open('../new_data/code_indices/code_dict_pretrained.pkl', 'wb') as f:
    pickle.dump(code2idx, f)

In [99]:
with open('../new_data/code_indices/code_dict_pretrained.pkl', 'rb') as f:
    code2idx = pickle.load(f)

In [100]:
data_dict_idx = data_dict.copy()

In [101]:
for k, visit in tqdm(data_dict_idx.items()):
    visit['seq_idx'] = [list(map(code2idx.get, v)) for v in visit['seq']]

100%|██████████| 39802/39802 [00:05<00:00, 6773.33it/s] 


In [102]:
# for k, visit in tqdm(data_dict_idx.items()):
#     visit['total_seq_idx'] = list(map(code2idx.get, visit['total_seq']))

In [103]:
data_dict_new= dict()
rm_patient_num =0
rm_patient_list = []
for p_id in tqdm(list(data_dict_idx.keys())):
    p_data = {'visit':[], 'visit_idx': [], 'visit_length':0, 'age': [], 'gender': None, 'race': None, 'admission_type': [], \
              'diagnoses_top100': None, 'diagnoses_top100_len': None, 'seq':None, 'seq_idx':None, 'timedelta': None, 'label':[]}
    
    for v_id, v_data in enumerate(data_dict_idx[p_id]['data']):            
        p_data['visit'].append(v_data['visit_id'])
        p_data['visit_idx'].append(v_id)
        p_data['seq'] = data_dict_idx[p_id]['seq']
        p_data['seq_idx'] = data_dict_idx[p_id]['seq_idx']
        p_data['time'] = data_dict_idx[p_id]['time']
        p_data['timedelta'] = data_dict_idx[p_id]['timedelta']
        p_data['age'].append(v_data['age'])
        p_data['admission_type'].append(v_data['admission_type'])
    
    p_data['diagnoses_top100'] = data_dict_idx[p_id]['diagnoses_top100']
    p_data['diagnoses_top100_len'] = data_dict_idx[p_id]['diagnoses_top100_len']
    assert len(p_data['diagnoses_top100']) == len(p_data['diagnoses_top100_len']) == len(p_data['visit'])
    p_data['race'] = data_dict_idx[p_id]['race']
    p_data['gender'] = data_dict_idx[p_id]['gender']
    p_data['label_top100'] = np.squeeze(data_dict_idx[p_id]['labels_top100_bin'])
    p_data['label_top10'] = np.squeeze(data_dict_idx[p_id]['labels_top10_bin'])
    p_data['label_top20'] = np.squeeze(data_dict_idx[p_id]['labels_top20_bin'])
    p_data['label_top40'] = np.squeeze(data_dict_idx[p_id]['labels_top40_bin'])
    p_data['label_top60'] = np.squeeze(data_dict_idx[p_id]['labels_top60_bin'])
    p_data['label_top80'] = np.squeeze(data_dict_idx[p_id]['labels_top80_bin'])
    p_data['label_bot10'] = np.squeeze(data_dict_idx[p_id]['labels_bot10_bin'])
    p_data['label_bot20'] = np.squeeze(data_dict_idx[p_id]['labels_bot20_bin'])
    p_data['label_bot40'] = np.squeeze(data_dict_idx[p_id]['labels_bot40_bin'])
    p_data['label_bot60'] = np.squeeze(data_dict_idx[p_id]['labels_bot60_bin'])
    p_data['label_bot80'] = np.squeeze(data_dict_idx[p_id]['labels_bot80_bin'])
    data_dict_new[p_id] = p_data

100%|██████████| 39802/39802 [00:00<00:00, 50188.31it/s]


In [104]:
with open('../new_data/data4pretrained_filter_drugs_preprocess_add_multibin.pkl', 'wb') as f:
    pickle.dump(data_dict_new, f)

In [105]:
with open('../new_data/data4pretrained_filter_drugs_preprocess_add_multibin.pkl', 'rb') as f:
    data_dict_new = pickle.load(f)

In [106]:
len(data_dict_new)

39802

In [107]:
data_dict_max50 = dict()
for p_id , data in data_dict_new.items():
    new_data = dict()
    if len(data['visit']) > 50:
        print(p_id, len(data['visit']))
        new_visit_idx = [i for i in range(len(data['visit_idx'][-50:]))]
    else:
        new_visit_idx = data['visit_idx']
    new_data['visit'] = data['visit'][-50:]
    new_data['visit_idx'] = new_visit_idx
    new_data['visit_length'] = len(data['visit'][-50:])
    new_data['seq'] = data['seq'][-50:]
    new_data['seq_idx'] = data['seq_idx'][-50:]
    new_data['time'] = data['time'][-50:]
    new_data['timedelta'] = data['timedelta'][-50:]
    new_data['age'] = data['age'][-50:]
    new_data['diagnoses_top100'] = data['diagnoses_top100'][-50:]
    new_data['diagnoses_top100_len'] = data['diagnoses_top100_len'][-50:]
    assert len(data['visit_idx'][-50:]) == len(data['seq'][-50:]) == len(data['age'][-50:]) == len(data['diagnoses_top100'][-50:])
    new_data['gender'] = data['gender']
    new_data['race'] = data['race']
    new_data['label_top100'] = data['label_top100']
    new_data['label_top10'] = data['label_top10']
    new_data['label_top20'] = data['label_top20']
    new_data['label_top40'] = data['label_top40']
    new_data['label_top60'] = data['label_top60']
    new_data['label_top80'] = data['label_top80']
    new_data['label_bot10'] = data['label_bot10']
    new_data['label_bot20'] = data['label_bot20']
    new_data['label_bot40'] = data['label_bot40']
    new_data['label_bot60'] = data['label_bot60']
    new_data['label_bot80'] = data['label_bot80']
    data_dict_max50[p_id] = new_data

10123949 55
10264646 76
10577647 78
10578325 67
10580201 59
10714009 113
10913302 64
11021643 63
11296936 76
11553072 69
11582633 81
11714071 62
11761621 56
11818101 64
11890447 66
12251785 79
12468016 88
12547294 69
12563258 60
12596559 63
13166511 53
13297743 91
13470788 52
13475033 96
13877234 60
13999829 54
14029474 52
14394983 124
15084163 53
15107347 51
15108590 52
15114531 67
15229574 79
15464144 75
15496609 133
15935768 56
16124481 62
16233333 67
16439884 58
16615356 55
16662316 129
16675371 66
16809525 57
16924675 54
17011846 56
17051420 54
17204468 59
17340686 51
17477304 51
17517983 94
17716210 80
17937834 61
18001923 69
18284271 82
18376342 52
18553055 55
18655830 69
18676703 59
18970086 68
19127408 51
19610016 52
19713100 62
19759225 63
19921471 57


In [108]:
with open('../new_data/data4pretrained_filter_drugs_preprocess_add_multibin_maxlen50.pkl', 'wb') as f:
    pickle.dump(data_dict_max50, f)

In [2]:
with open('../new_data/data4pretrained_filter_drugs_preprocess_add_multibin_maxlen50.pkl', 'rb') as f:
    data_dict_max50 = pickle.load(f)

In [3]:
len(data_dict_max50)

39802

In [2]:
def convert_code_type(code):
    if code.startswith('d'):
        return 1
    elif code.startswith('pcs'):
        return 2
    elif code.startswith('p_'):
        return 3
    elif code in ['[CLS]', '[SEP]']:
        return 4

In [111]:
from sklearn.model_selection import train_test_split

In [112]:
seed_list = [123, 321, 666, 777, 5959]

In [125]:
total_patient_list = list(data_dict_max50.keys())

In [126]:
# for seed in seed_list:
#     train_indices, test_indices = train_test_split(total_patient_list, test_size=0.1, random_state=seed)
#     train_indices, val_indices = train_test_split(train_indices, test_size=len(test_indices), random_state=seed)
#     print(len(train_indices), len(val_indices), len(test_indices))
#     with open(f'../new_data/split_indices/train_indices_{seed}.pkl', 'wb') as f:
#         pickle.dump(train_indices, f)
#     with open(f'../new_data/split_indices/val_indices_{seed}.pkl', 'wb') as f:
#         pickle.dump(val_indices, f)
#     with open(f'../new_data/split_indices/test_indices_{seed}.pkl', 'wb') as f:
#         pickle.dump(test_indices, f)

In [113]:
slide_length = 5

In [114]:
for seed in seed_list:
    with open(f'../new_data/split_indices/train_indices_{seed}.pkl', 'rb') as f:
        train_indices = pickle.load(f)
    f.close()
    
    total_len_list = []
    train_datasets = dict()
    for p_id in tqdm(train_indices):
        data = data_dict_max50[p_id]
        if len(data['visit']) > slide_length:
            for i in range(len(data['visit']) - slide_length + 1):
                visit_data = dict()
                visit_data['patient_id'] = p_id
                visit_data['visit'] = data['visit'][i:i+slide_length]
                visit_data['visit_idx'] = data['visit_idx'][i:i+slide_length]
                visit_data['visit_length'] = len(data['visit'][i:i+slide_length])
                
                slide_seq = data['seq'][i:i+slide_length]
                slide_seq_idx = data['seq_idx'][i:i+slide_length]
                
                total_seq = ['[CLS]'] + [item for sublist in slide_seq for item in sublist + ['[SEP]']]
                total_seq_idx = [code2idx['[CLS]']] + [item for sublist in slide_seq_idx for item in sublist + [code2idx['[SEP]']]]
                total_code_type = list(map(convert_code_type, total_seq))
                total_vis_seg = [visit_data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(visit_data['visit_idx']) for elem in [v_idx] * len(slide_seq_idx[idx]) + [v_idx]]
                assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)
                
                visit_data['total_seq'] = total_seq
                visit_data['total_seq_idx'] = total_seq_idx
                visit_data['code_type'] = total_code_type
                visit_data['visit_seg'] = total_vis_seg
                visit_data['total_len'] = len(total_seq)
                visit_data['time'] = data['time'][i:i+slide_length]

                timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'][i:i+slide_length])}
                visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
                timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                                    for idx, tm in enumerate(data['time'][i:i+slide_length])}
                visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
                if i == len(data['visit']) - slide_length :
                    visit_data['label_top10'] = data['label_top10']
                    visit_data['label_top20'] = data['label_top20']
                    visit_data['label_top40'] = data['label_top40']
                    visit_data['label_top60'] = data['label_top60']
                    visit_data['label_top80'] = data['label_top80']
                    visit_data['label_top100'] = data['label_top100']
                    visit_data['label_bot10'] = data['label_bot10']
                    visit_data['label_bot20'] = data['label_bot20']
                    visit_data['label_bot40'] = data['label_bot40']
                    visit_data['label_bot60'] = data['label_bot60']
                    visit_data['label_bot80'] = data['label_bot80']
                    visit_data['label_visit_idx'] = data['visit_idx'][-1] + 1
                else:
                    visit_data['label_top10'] = mlb_t10.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_top20'] = mlb_t20.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_top40'] = mlb_t40.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_top60'] = mlb_t60.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_top80'] = mlb_t80.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_top100'] = mlb_t100.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_bot10'] = mlb_b10.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_bot20'] = mlb_b20.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_bot40'] = mlb_b40.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_bot60'] = mlb_b60.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_bot80'] = mlb_b80.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_visit_idx'] = data['visit_idx'][i+slide_length]
                train_datasets[f'{str(p_id)}_slide{str(i)}'] = visit_data
        else:
            visit_data = dict()
            visit_data['patient_id'] = p_id
            visit_data['visit'] = data['visit']
            visit_data['visit_idx'] = data['visit_idx']
            visit_data['visit_length'] = data['visit_length']
            
            total_seq = ['[CLS]'] + [item for sublist in data['seq'] for item in sublist + ['[SEP]']]
            total_seq_idx = [code2idx['[CLS]']] + [item for sublist in data['seq_idx'] for item in sublist + [code2idx['[SEP]']]]
            total_code_type = list(map(convert_code_type, total_seq))
            total_vis_seg = [data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(data['visit_idx']) for elem in [v_idx] * len(data['seq_idx'][idx]) + [v_idx]]
            assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)
            
            visit_data['total_seq'] = total_seq
            visit_data['total_seq_idx'] = total_seq_idx
            visit_data['code_type'] = total_code_type
            visit_data['visit_seg'] = total_vis_seg
            visit_data['total_len'] = len(total_seq)
            visit_data['time'] = data['time']
            
            timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'])}
            visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
            timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                                for idx, tm in enumerate(data['time'])}
            visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
            
            visit_data['label_top10'] = data['label_top10']
            visit_data['label_top20'] = data['label_top20']
            visit_data['label_top40'] = data['label_top40']
            visit_data['label_top60'] = data['label_top60']
            visit_data['label_top80'] = data['label_top80']
            visit_data['label_top100'] = data['label_top100']
            visit_data['label_bot10'] = data['label_bot10']
            visit_data['label_bot20'] = data['label_bot20']
            visit_data['label_bot40'] = data['label_bot40']
            visit_data['label_bot60'] = data['label_bot60']
            visit_data['label_bot80'] = data['label_bot80']

            visit_data['label_visit_idx'] = data['visit_idx'][-1] + 1
            train_datasets[f'{str(p_id)}_slide{str(0)}'] = visit_data
            
        total_len_list.append(len(total_seq))
    
    print(max(total_len_list))
    with open(f'../new_data/split_datasets/train_datasets_{seed}_multi_labels.pkl', 'wb') as f:
        pickle.dump(train_datasets, f)

100%|██████████| 31840/31840 [00:34<00:00, 930.87it/s] 


395


100%|██████████| 31840/31840 [00:35<00:00, 902.42it/s] 


395


100%|██████████| 31840/31840 [00:29<00:00, 1062.44it/s]


395


100%|██████████| 31840/31840 [00:34<00:00, 920.01it/s] 


381


100%|██████████| 31840/31840 [00:35<00:00, 901.75it/s] 


395


In [115]:
data.keys()

dict_keys(['visit', 'visit_idx', 'visit_length', 'seq', 'seq_idx', 'time', 'timedelta', 'age', 'diagnoses_top100', 'diagnoses_top100_len', 'gender', 'race', 'label_top100', 'label_top10', 'label_top20', 'label_top40', 'label_top60', 'label_top80', 'label_bot10', 'label_bot20', 'label_bot40', 'label_bot60', 'label_bot80'])

In [116]:
for seed in seed_list:
    with open(f'../new_data/split_indices/val_indices_{seed}.pkl', 'rb') as f:
        val_indices = pickle.load(f)
    f.close()
    
    total_len_list = []
    valid_datasets = dict()
    for p_id in tqdm(val_indices):
        data = data_dict_max50[p_id]
        visit_data = dict()
        visit_data['patient_id'] = p_id
        visit_data['visit'] = data['visit'][-slide_length:]
        visit_data['visit_idx'] = data['visit_idx'][-slide_length:]
        visit_data['visit_length'] = len(data['visit'][-slide_length:])

        slide_seq = data['seq'][-slide_length:]
        slide_seq_idx = data['seq_idx'][-slide_length:]

        total_seq = ['[CLS]'] + [item for sublist in slide_seq for item in sublist + ['[SEP]']]
        total_seq_idx = [code2idx['[CLS]']] + [item for sublist in slide_seq_idx for item in sublist + [code2idx['[SEP]']]]
        total_code_type = list(map(convert_code_type, total_seq))
        total_vis_seg = [visit_data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(visit_data['visit_idx']) for elem in [v_idx] * len(slide_seq_idx[idx]) + [v_idx]]
        assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)

        visit_data['total_seq'] = total_seq
        visit_data['total_seq_idx'] = total_seq_idx
        visit_data['code_type'] = total_code_type
        visit_data['visit_seg'] = total_vis_seg
        visit_data['total_len'] = len(total_seq)
        visit_data['time'] = data['time'][-slide_length:]

        timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'][-slide_length:])}
        visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
        timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                            for idx, tm in enumerate(data['time'][-slide_length:])}
        visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
        # visit_data['label'] = data['label']
        
        visit_data['label_top10'] = data['label_top10']
        visit_data['label_top20'] = data['label_top20']
        visit_data['label_top40'] = data['label_top40']
        visit_data['label_top60'] = data['label_top60']
        visit_data['label_top80'] = data['label_top80']
        visit_data['label_top100'] = data['label_top100']
        visit_data['label_bot10'] = data['label_bot10']
        visit_data['label_bot20'] = data['label_bot20']
        visit_data['label_bot40'] = data['label_bot40']
        visit_data['label_bot60'] = data['label_bot60']
        visit_data['label_bot80'] = data['label_bot80']
        
        visit_data['label_visit_idx'] = data['visit_idx'][-1] + 1
        valid_datasets[f'{str(p_id)}_slide{str(0)}'] = visit_data
        total_len_list.append(len(total_seq))
    print(max(total_len_list))
    with open(f'../new_data/split_datasets/valid_datasets_{seed}_multi_labels.pkl', 'wb') as f:
        pickle.dump(valid_datasets, f)

100%|██████████| 3981/3981 [00:00<00:00, 10668.95it/s]


367


100%|██████████| 3981/3981 [00:00<00:00, 10696.12it/s]


386


100%|██████████| 3981/3981 [00:00<00:00, 11019.68it/s]


371


100%|██████████| 3981/3981 [00:00<00:00, 10793.34it/s]


395


100%|██████████| 3981/3981 [00:00<00:00, 10914.19it/s]


384


In [117]:
for seed in seed_list:
    with open(f'../new_data/split_indices/test_indices_{seed}.pkl', 'rb') as f:
        test_indices = pickle.load(f)
    f.close()
    
    total_len_list = []
    test_datasets = dict()
    for p_id in tqdm(test_indices):
        data = data_dict_max50[p_id]
        visit_data = dict()
        visit_data['patient_id'] = p_id
        visit_data['visit'] = data['visit'][-slide_length:]
        visit_data['visit_idx'] = data['visit_idx'][-slide_length:]
        visit_data['visit_length'] = len(data['visit'][-slide_length:])

        slide_seq = data['seq'][-slide_length:]
        slide_seq_idx = data['seq_idx'][-slide_length:]

        total_seq = ['[CLS]'] + [item for sublist in slide_seq for item in sublist + ['[SEP]']]
        total_seq_idx = [code2idx['[CLS]']] + [item for sublist in slide_seq_idx for item in sublist + [code2idx['[SEP]']]]
        total_code_type = list(map(convert_code_type, total_seq))
        total_vis_seg = [visit_data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(visit_data['visit_idx']) for elem in [v_idx] * len(slide_seq_idx[idx]) + [v_idx]]
        assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)

        visit_data['total_seq'] = total_seq
        visit_data['total_seq_idx'] = total_seq_idx
        visit_data['code_type'] = total_code_type
        visit_data['visit_seg'] = total_vis_seg
        visit_data['total_len'] = len(total_seq)
        visit_data['time'] = data['time'][-slide_length:]

        timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'][-slide_length:])}
        visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
        timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                            for idx, tm in enumerate(data['time'][-slide_length:])}
        visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
        # visit_data['label'] = data['label']
        
        visit_data['label_top10'] = data['label_top10']
        visit_data['label_top20'] = data['label_top20']
        visit_data['label_top40'] = data['label_top40']
        visit_data['label_top60'] = data['label_top60']
        visit_data['label_top80'] = data['label_top80']
        visit_data['label_top100'] = data['label_top100']
        visit_data['label_bot10'] = data['label_bot10']
        visit_data['label_bot20'] = data['label_bot20']
        visit_data['label_bot40'] = data['label_bot40']
        visit_data['label_bot60'] = data['label_bot60']
        visit_data['label_bot80'] = data['label_bot80']
        visit_data['label_visit_idx'] = data['visit_idx'][-1] + 1
        
        test_datasets[f'{str(p_id)}_slide{str(0)}'] = visit_data
        total_len_list.append(len(total_seq))
        
    print(max(total_len_list))
    with open(f'../new_data/split_datasets/test_datasets_{seed}_multi_labels.pkl', 'wb') as f:
        pickle.dump(test_datasets, f)

100%|██████████| 3981/3981 [00:00<00:00, 10991.08it/s]


380


100%|██████████| 3981/3981 [00:00<00:00, 10899.62it/s]


380


100%|██████████| 3981/3981 [00:00<00:00, 11046.13it/s]


365


100%|██████████| 3981/3981 [00:00<00:00, 11026.59it/s]


391


100%|██████████| 3981/3981 [00:00<00:00, 10797.92it/s]


381


In [119]:
min(total_len_list)

7

In [1]:
863 + 738 + 3487

5088